In [8]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from bs4 import BeautifulSoup
import scrapy
import pandas as pd
import csv
import os
import requests
import time
import random 
from datetime import datetime


In [9]:
driver = webdriver.Chrome('./chromedriver.exe')
driver.get('https://ebird.org/explore')
input_ebird = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"input[placeholder='Enter species name']")))


C:\Users\LEGION\AppData\Local\Temp\ipykernel_37572\4268123770.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver.exe')


In [10]:
input_ebird.send_keys('Mourning dove')
input_ebird.send_keys(Keys.ENTER)
time.sleep(2)

In [11]:
input_ebird.send_keys(Keys.ENTER)

In [12]:
input_ebird.send_keys(Keys.ENTER)

In [ ]:

# placeholder="Enter species name" 
def get_page_of_bird_ebird(bird_name):
    driver.get('https://ebird.org/explore')
    input_eol = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"input[name='q']")))
    input_eol.send_keys(bird_name)
    input_eol.send_keys(u'\ue007')
    try:
        ps = driver.find_element(by=By.CLASS_NAME, value="uk-link-reset")
        return ps.get_attribute('href')
    except Exception:
        return None

def scrape_v_2_ebird(URL):
    if URL :
        d = {}
        page = requests.get(URL)
        soup = BeautifulSoup(page.content, "html.parser")
        keys = soup.find_all("div",class_="sample-trait-key")
        values = soup.find_all("div",class_="sample-trait-val")
        
        for i,j in zip(keys,values):
            key = i.get_text()
            value = j.get_text()
            if key in d:
                d[key] = str(d[key]) + "," + str(value)
            else:
                d[key] = value
        return d
    return 'NODATA'


def ebird(fro,to,csv_file):
    types_of_birds_df = pd.read_csv('assets/birds.csv',index_col='ID')
    list_of_bird_names = [ types_of_birds_df.loc[ _ ,'Common Bird Names'] for _ in range(1,types_of_birds_df.shape[0]) ]
    data_from_eol = {}
    re_search_on = [] 
    writer = csv.writer(open('assets/'+ csv_file +'.csv',"a",newline=''))
    for bird_name in list_of_bird_names[fro:to]:
        try:
            data_from_eol[bird_name] = scrape_v_2_ebird(get_page_of_bird_ebird(bird_name))
            try:
                writer.writerow([bird_name,data_from_eol[bird_name]])
            except UnicodeEncodeError:
                writer.writerow([list_of_bird_names.index(bird_name),data_from_eol[bird_name]])
            if (not data_from_eol) or (data_from_eol[bird_name]=='NODATA') :
                re_search_on.append(bird_name)
                print('[-] ',bird_name)
            else:
                print('[+] ',bird_name)
        except Exception:
            writer.writerow([bird_name,'NODATA'])
            print('----------> Unknown Exceptions for Bird ',bird_name)
    print(re_search_on)

driver = webdriver.Chrome('./chromedriver.exe')